Les Trois Types d'Attention et l'Attention par Produit Scalaire : Cahier de Laboratoire Non Évalué

Dans ce Notebook , vous explorerez les trois types d'attention (attention encodeur-décodeur, attention causale et attention self-attention bidirectionnelle) et comment implémenter les deux derniers avec l'attention par produit scalaire.

<img src="https://github.com/amanjeetsahu/Natural-Language-Processing-Specialization/blob/master/Natural%20Language%20Processing%20with%20Attention%20Models/Week%202/attention_lnb_figs/C4_W2_L3_dot-product-attention_S01_introducing-attention_stripped.png?raw=1" width="500"/>

Cette semaine, vous découvrirez comment intégrer l'attention dans les **transformers**. Parce que les transformers ne sont pas des modèles de séquence, ils sont beaucoup plus faciles à paralléliser et à accélérer. En plus de la traduction automatique, les applications des transformers comprennent :
* L'auto-complétion
* La reconnaissance d'entités nommées
* Les chatbots
* Les questions-réponses
* Et bien plus encore !

En plus de l'intégration, du codage de position, des couches denses et des connexions résiduelles, l'attention est un composant essentiel des transformers. Au cœur de tout schéma d'attention utilisé dans un transformer se trouve l'**attention par produit scalaire**, dont les figures ci-dessous présentent une image simplifiée :

<img src="https://github.com/amanjeetsahu/Natural-Language-Processing-Specialization/blob/master/Natural%20Language%20Processing%20with%20Attention%20Models/Week%202/attention_lnb_figs/C4_W2_L3_dot-product-attention_S03_concept-of-attention_stripped.png?raw=1" width="500"/>

<img src="https://github.com/amanjeetsahu/Natural-Language-Processing-Specialization/blob/master/Natural%20Language%20Processing%20with%20Attention%20Models/Week%202/attention_lnb_figs/C4_W2_L3_dot-product-attention_S04_attention-math_stripped.png?raw=1" width="500"/>

Avec l'attention de base par produit scalaire, vous capturez les interactions entre chaque mot (incorporation) de votre requête et chaque mot de votre clé. Si les requêtes et les clés appartiennent aux mêmes phrases, cela constitue une **auto-attention bidirectionnelle**. Cependant, dans certaines situations, il est plus approprié de considérer uniquement les mots qui précèdent le mot actuel. De tels cas, en particulier lorsque les requêtes et les clés proviennent des mêmes phrases, entrent dans la catégorie de l'**attention causale**.

<img src="https://github.com/amanjeetsahu/Natural-Language-Processing-Specialization/blob/master/Natural%20Language%20Processing%20with%20Attention%20Models/Week%202/attention_lnb_figs/C4_W2_L4_causal-attention_S02_causal-attention_stripped.png?raw=1" width="500"/>

Pour l'attention causale, nous ajoutons un **masque** à l'argument de notre fonction softmax, comme illustré ci-dessous :

<img src="https://github.com/amanjeetsahu/Natural-Language-Processing-Specialization/blob/master/Natural%20Language%20Processing%20with%20Attention%20Models/Week%202/attention_lnb_figs/C4_W2_L4_causal-attention_S03_causal-attention-math_stripped.png?raw=1" width="500"/>

<img src="https://github.com/amanjeetsahu/Natural-Language-Processing-Specialization/blob/master/Natural%20Language%20Processing%20with%20Attention%20Models/Week%202/attention_lnb_figs/C4_W2_L4_causal-attention_S04_causal-attention-math-2_stripped.png?raw=1" width="500"/>

Maintenant, voyons comment implémenter l'attention avec NumPy. Lorsque vous intégrez l'attention dans un réseau transformer défini avec Trax, vous devrez utiliser `trax.fastmath.numpy` à la place, car les tableaux de Trax sont basés sur les DeviceArrays de JAX. Heureusement, les interfaces des fonctions sont souvent identiques.

## Imports

In [ ]:
import sys

import numpy as np
import scipy.special

import textwrap
wrapper = textwrap.TextWrapper(width=70)

# to print the entire np array
np.set_printoptions(threshold=sys.maxsize)

Voici quelques fonctions auxiliaires qui vous aideront à créer des tenseurs et à afficher des informations utiles :

* `create_tensor()` crée un tableau numpy à partir d'une liste de listes.
* `display_tensor()` affiche la forme et le tenseur réel.

In [ ]:
def create_tensor(t):
    """Create tensor from list of lists"""
    return np.array(t)


def display_tensor(t, name):
    """Display shape and tensor"""
    print(f'{name} shape: {t.shape}\n')
    print(f'{t}\n')

Créez quelques tenseurs et affichez leurs formes. N'hésitez pas à expérimenter avec vos propres tenseurs. Gardez à l'esprit, cependant, que les tableaux de requêtes (query), de clés (key) et de valeurs (value) doivent tous avoir les mêmes dimensions d'intégration (nombre de colonnes), et le tableau de masque (mask) doit avoir la même forme que `np.dot(query, key.T)`.

In [ ]:
q = create_tensor([[1, 0, 0], [0, 1, 0]])
display_tensor(q, 'query')
k = create_tensor([[1, 2, 3], [4, 5, 6]])
display_tensor(k, 'key')
v = create_tensor([[0, 1, 0], [1, 0, 1]])
display_tensor(v, 'value')
m = create_tensor([[0, 0], [-1e9, 0]])
display_tensor(m, 'mask')

query shape: (2, 3)

[[1 0 0]
 [0 1 0]]

key shape: (2, 3)

[[1 2 3]
 [4 5 6]]

value shape: (2, 3)

[[0 1 0]
 [1 0 1]]

mask shape: (2, 2)

[[ 0.e+00  0.e+00]
 [-1.e+09  0.e+00]]



## Attention à produit scalaire

Nous arrivons maintenant à l'essentiel de ce laboratoire, où nous calculons $\textrm{softmax} \left(\frac{Q K^T}{\sqrt{d}} + M \right) V$, où le facteur d'échelle (optionnel mais par défaut) $\sqrt{d}$ est la racine carrée de la dimension d'intégration.

In [ ]:
def DotProductAttention(query, key, value, mask, scale=True):
    """Dot product self-attention.
    Args:
        query (numpy.ndarray): array of query representations with shape (L_q by d)
        key (numpy.ndarray): array of key representations with shape (L_k by d)
        value (numpy.ndarray): array of value representations with shape (L_k by d) where L_v = L_k
        mask (numpy.ndarray): attention-mask, gates attention with shape (L_q by L_k)
        scale (bool): whether to scale the dot product of the query and transposed key

    Returns:
        numpy.ndarray: Self-attention array for q, k, v arrays. (L_q by L_k)
    """

    assert query.shape[-1] == key.shape[-1] == value.shape[-1], "Embedding dimensions of q, k, v aren't all the same"

    # Save depth/dimension of the query embedding for scaling down the dot product
    if scale:
        depth = query.shape[-1]
    else:
        depth = 1

    # Calculate scaled query key dot product according to formula above
    dots = np.matmul(query, np.swapaxes(key, -1, -2)) / np.sqrt(depth)

    # Apply the mask
    if mask is not None:
        dots = np.where(mask, dots, np.full_like(dots, -1e9))

    # Softmax formula implementation
    # Use scipy.special.logsumexp of masked_qkT to avoid underflow by division by large numbers
    # Note: softmax = e^(dots - logaddexp(dots)) = E^dots / sumexp(dots)
    logsumexp = scipy.special.logsumexp(dots, axis=-1, keepdims=True)

    # Take exponential of dots minus logsumexp to get softmax
    # Use np.exp()
    dots = np.exp(dots - logsumexp)

    # Multiply dots by value to get self-attention
    # Use np.matmul()
    attention = np.matmul(dots, value)

    return attention

Now let's implement the *masked* dot product self-attention (at the heart of causal attention) as a special case of dot product attention

In [ ]:
def dot_product_self_attention(q, k, v, scale=True):
    """ Masked dot product self attention.
    Args:
        q (numpy.ndarray): queries.
        k (numpy.ndarray): keys.
        v (numpy.ndarray): values.
    Returns:
        numpy.ndarray: masked dot product self attention tensor.
    """

    # Size of the penultimate dimension of the query
    mask_size = q.shape[-2]

    # Creates a matrix with ones below the diagonal and 0s above. It should have shape (1, mask_size, mask_size)
    # Use np.tril() - Lower triangle of an array and np.ones()
    mask = np.tril(np.ones((1, mask_size, mask_size), dtype=np.bool_), k=0)

    return DotProductAttention(q, k, v, mask, scale=scale)

In [ ]:
dot_product_self_attention(q, k, v)